# **Upload and Run the Below Code in Google Collab**

In [ ]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

# **Import the Required Models**

In [ ]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

# **Add the Downloaded dataset to your google drives `My Drive` Section. Then use Below Code to connect it to this Project**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
my_folder_path = '/content/drive/MyDrive/sign_language_dataset'

In [ ]:
print(my_folder_path)

In [ ]:
print(my_folder_path)
labels = []
for i in os.listdir(my_folder_path):
  if os.path.isdir(os.path.join(my_folder_path, i)):
    labels.append(i)
for label in labels:
  print(label + "\n")

In [ ]:
print(len(labels))

# **From Below Run All the Steps to Train and then Download the Trained Model**

In [ ]:
NUM_EXAMPLES = 10

for label in labels:
  label_dir = os.path.join(my_folder_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

<ipython-input-9-fbc998dd906b>:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))


In [ ]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=my_folder_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

In [ ]:
hparams = gesture_recognizer.HParams(export_dir="exported_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer)  [(None, 128)]            0         
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_o  (None, 42)               5418      
 ut (Dense)                                                      
                                                                 
Total params: 5,930
Trainable params: 5,674
Non-trainable par

In [ ]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

186/186 [==============================] - 4s 4ms/step - loss: 0.3927 - categorical_accuracy: 0.8333
Test loss:0.3927353620529175, Test accuracy:0.8333333134651184


In [ ]:
model.export_model()
!ls exported_model

best_model_weights.data-00000-of-00001	epoch_models		 metadata.json
best_model_weights.index		gesture_recognizer.task
checkpoint				logs


In [ ]:
files.download('exported_model/gesture_recognizer.task')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>